<h1 style="text-align:center; margin-bottom:20px;">
Forward and Backward propagation of a 2-layer neural network
</h1>


## Network Definition
$$
z_1 = XW_1 + b_1,\quad a_1 = \mathrm{ReLU}(z_1)
$$
$$
z_2 = a_1W_2 + b_2,\quad \hat{Y} = \mathrm{softmax}(z_2)
$$
$$
\mathcal{L} = -\frac{1}{B}\sum_{i=1}^B \log(\hat{y}_{i,y_i})
$$



## Objective
Compute gradients:
$$
\frac{\partial \mathcal{L}}{\partial W_2},\;
\frac{\partial \mathcal{L}}{\partial b_2},\;
\frac{\partial \mathcal{L}}{\partial W_1},\;
\frac{\partial \mathcal{L}}{\partial b_1}
$$



## Step 1 — Softmax + Cross-Entropy

Softmax:
$$
\hat{y}_k = \frac{e^{z_k}}{\sum_j e^{z_j}}
$$

Cross-Entropy:
$$
\mathcal{L} = -\sum_k y_k \log \hat{y}_k
$$

Gradient simplification:
$$
\boxed{
\frac{\partial \mathcal{L}}{\partial z_2}
=
\frac{1}{B}(\hat{Y} - Y)
}
$$


## Step 2 — Output Linear Layer
$$
z_2 = a_1W_2 + b_2
$$

Weight gradient:
$$
\boxed{
\frac{\partial \mathcal{L}}{\partial W_2}
=
a_1^T \frac{\partial \mathcal{L}}{\partial z_2}
}
$$

Bias gradient:
$$
\boxed{
\frac{\partial \mathcal{L}}{\partial b_2}
=
\sum_{i=1}^B \frac{\partial \mathcal{L}}{\partial z_2^{(i)}}
}
$$

Hidden activation gradient:
$$
\boxed{
\frac{\partial \mathcal{L}}{\partial a_1}
=
\frac{\partial \mathcal{L}}{\partial z_2} W_2^T
}
$$



## Step 3 — ReLU Nonlinearity
$$
a_1 = \max(0, z_1)
$$

Derivative:
$$
\mathrm{ReLU}'(z_1) =
\begin{cases}
1 & z_1 > 0 \\
0 & z_1 \le 0
\end{cases}
$$

Chain rule:
$$
\boxed{
\frac{\partial \mathcal{L}}{\partial z_1}
=
\frac{\partial \mathcal{L}}{\partial a_1}
\odot \mathbb{1}(z_1 > 0)
}
$$



## Step 4 — First Linear Layer
$$
z_1 = XW_1 + b_1
$$

Weight gradient:
$$
\boxed{
\frac{\partial \mathcal{L}}{\partial W_1}
=
X^T \frac{\partial \mathcal{L}}{\partial z_1}
}
$$

Bias gradient:
$$
\boxed{
\frac{\partial \mathcal{L}}{\partial b_1}
=
\sum_{i=1}^B \frac{\partial \mathcal{L}}{\partial z_1^{(i)}}
}
$$



## Final Collapsed Gradient Chain
$$
\boxed{
\frac{\partial \mathcal{L}}{\partial W_1}
=
X^T
\Big[
\big((\hat{Y} - Y) W_2^T\big)
\odot \mathbb{1}(z_1 > 0)
\Big]
}
$$

$$
\boxed{
\frac{\partial \mathcal{L}}{\partial W_2}
=
a_1^T (\hat{Y} - Y)
}
$$




In [ ]:
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

# Load MNIST data
print("Loading MNIST...")
X, y = fetch_openml('mnist_784', version=1, return_X_y=True, as_frame=False, cache=True)

# Normalize and prepare data
X = X.astype(np.float32) / 255.0
y = y.astype(np.int64)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=60000, test_size=10000, random_state=42, stratify=y
)

# Initialize network weights
np.random.seed(42)
W1 = np.random.randn(784, 128) / np.sqrt(784)
b1 = np.zeros((1, 128))
W2 = np.random.randn(128, 10) / np.sqrt(128)
b2 = np.zeros((1, 10))

# Hyperparameters
lr = 0.1
epochs = 20
batch_size = 64

# Activation functions
def relu(z):
    return np.maximum(0, z)

def softmax(z):
    e = np.exp(z - np.max(z, axis=1, keepdims=True))
    return e / np.sum(e, axis=1, keepdims=True)

# Training loop
for epoch in range(epochs):
    # Shuffle data
    idx = np.random.permutation(len(X_train))
    X_shuf, y_shuf = X_train[idx], y_train[idx]
    
    losses = []
    
    for i in range(0, len(X_train), batch_size):
        Xb = X_shuf[i:i+batch_size]
        yb = y_shuf[i:i+batch_size]
        B = len(Xb)
        
        # Forward pass
        z1 = Xb @ W1 + b1
        a1 = relu(z1)
        z2 = a1 @ W2 + b2
        probs = softmax(z2)
        
        # Compute loss
        log_probs = np.log(probs[range(B), yb] + 1e-10)
        loss = -np.mean(log_probs)
        losses.append(loss)
        
        # Backward pass
        dz2 = probs.copy()
        dz2[range(B), yb] -= 1
        dz2 /= B
        
        dW2 = a1.T @ dz2
        db2 = np.sum(dz2, axis=0, keepdims=True)
        
        da1 = dz2 @ W2.T
        dz1 = da1 * (z1 > 0)
        
        dW1 = Xb.T @ dz1
        db1 = np.sum(dz1, axis=0, keepdims=True)
        
        # Update weights
        W2 -= lr * dW2
        b2 -= lr * db2
        W1 -= lr * dW1
        b1 -= lr * db1
    
    print(f"Epoch {epoch+1:2d}/{epochs}   loss = {np.mean(losses):.4f}")

# Evaluate
z1 = X_test @ W1 + b1
a1 = relu(z1)
z2 = a1 @ W2 + b2
preds = np.argmax(softmax(z2), axis=1)

accuracy = (preds == y_test).mean() * 100
print(f"\nTest accuracy: {accuracy:.2f}%")

$$X∈RB×784$$